In [37]:
import pandas as pd
import pymongo
import sshtunnel
import os
import numpy as np

In [2]:
server_ip = open('server.ip', 'r').readline().strip()
database = 'Windows-DESKTOP-3HVBJEA'
collection_name = 'HARVEST_DATA'


In [3]:
def upload_csv_to_database(records):
    with sshtunnel.open_tunnel((server_ip, 2222),
                               ssh_username='andrew',
                               ssh_password='a',
                               remote_bind_address=('127.0.0.1', 27017),
                               local_bind_address=('127.0.0.1', 37017)
                               ) as tunnel:
        # Construct MongoDB URI with the local bind address port
        mongo_uri = f"mongodb://localhost:{37017}"

        # connect to mongo uri
        client = pymongo.MongoClient(mongo_uri)
        # get the database
        db = client[database]
        # get the collection
        collection = db[collection_name]
        # insert the data
        collection.insert_many(records)
        # Close the connection
        client.close()


In [4]:
# get all files in the directory
directory = 'fruitSM2'
files = os.listdir(directory)
files.sort()
files

['24.06.24.csv',
 '24.07.01.csv',
 '24.07.10.csv',
 '24.07.15.csv',
 '24.07.22.csv',
 '24.07.29.csv',
 '24.08.05.csv',
 '24.08.12.csv',
 '24.08.19.csv',
 '24.08.26.csv',
 '24.08.30.csv',
 '24.09.02.csv',
 '24.09.06.csv',
 '24.09.09.csv',
 '24.09.13.csv',
 '24.09.19.csv',
 '24.09.23.csv',
 '24.09.27.csv',
 '24.10.04.csv',
 '24.10.07.csv',
 '24.10.11.csv',
 '24.10.14.csv',
 '24.10.18.csv',
 '24.10.28.csv',
 '24.11.01.csv',
 '24.11.04.csv',
 '24.11.08.csv',
 '24.11.09.csv',
 '24.11.11.csv',
 '24.11.15.csv',
 '24.11.22.csv',
 '24.11.25.csv',
 '24.11.29.csv',
 '24.12.02.csv',
 '24.12.06.csv']

In [26]:
# harvest_format = { 
#     "Date": "", # Date format aka file name extracted
#     "Product Name": "", # '죽향저장', '죽향실온', '죽향(실온)', '메리퀸', '죽향(저장)'
#     "Section": "", 
#     "Plant": "",
#     "Fruit vertical length": "",
#     "Fruit horizontal length": "",
#     "Fruit weight": "",
#     "Hardness": "",
#     "Sweetness": "",
#     "Acidity": "",
#     "Color L": "",
#     "Color a": "",
#     "Color b": "",
#     "Remarks": "",
# }   

In [36]:
# dump all data into the database
for file in files:  # file format: 21.07.01.csv
    df = pd.read_csv(f'{directory}/{file}', encoding='utf-8')
    datetime = file[:-4]
    # convert to datetime format to store in the database
    datetime = f'20{datetime[:2]}-{datetime[3:5]}-{datetime[6:]}'
    # create new dataframe with new column date from df
    df['Date'] = datetime
    # move Date column to the first column
    cols = df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    df = df[cols]
    # convert dataframe to dictionary
    records = df.to_dict(orient='records')
    # upload to database
    upload_csv_to_database(records)
    print(f'{file} uploaded to database')


In [24]:
# # list all name of products name
# unique_products = []
# for file in files:
#     df = pd.read_csv(f'{directory}/{file}', encoding='utf-8')
#     products = df['Product Name'].unique()
#     for product in products:
#         if product not in unique_products:
#             print(file)
#             print(product)
#             unique_products.append(product)
#     
# unique_products = list(set(unique_products))

24.06.24.csv
메리퀸
24.06.24.csv
죽향(실온)
24.07.01.csv
죽향(저장)
24.07.15.csv
죽향실온
24.07.15.csv
죽향저장


['죽향저장', '죽향실온', '죽향(실온)', '메리퀸', '죽향(저장)']


# Query the database

In [ ]:
def query_database():
    with sshtunnel.open_tunnel((server_ip, 2222),
                               ssh_username='andrew',
                               ssh_password='a',
                               remote_bind_address=('127.0.0.1', 27017),
                               local_bind_address=('127.0.0.1', 37017)
                               ) as tunnel:
        # Construct MongoDB URI with the local bind address port
        mongo_uri = f"mongodb://localhost:{37017}"

        # connect to mongo uri
        client = pymongo.MongoClient(mongo_uri)


In [52]:
from datetime import datetime, timedelta

with sshtunnel.open_tunnel((server_ip, 2222),
                           ssh_username='andrew',
                           ssh_password='a',
                           remote_bind_address=('127.0.0.1', 27017),
                           local_bind_address=('127.0.0.1', 37017)
                           ) as tunnel:
    # Construct MongoDB URI with the local bind address port
    mongo_uri = f"mongodb://localhost:{37017}"

    # connect to mongo uri
    client = pymongo.MongoClient(mongo_uri)

    db = client[database]

    # Collections
    com_collection = db["COM"]
    com4_collection = db["COM4"]
    com5_collection = db["COM5"]
    harvest_data_collection = db["HARVEST_DATA"]
    
    # Cache for sensor data to avoid repeated queries (keys are date and product name)
    sensor_data_cache = {}

    # Function to union data
    def union_data(aggregation="AVG"):
        result = []
        
        def custom_avg(row):
            valid_values = [value for value in row if not pd.isnull(value) and value != 0]
            return np.mean(valid_values) if valid_values else None

        # Iterate over HARVEST_DATA
        for harvest in harvest_data_collection.find():
            product_name = harvest["Product Name"]
            date = datetime.strptime(harvest["Date"], "%Y-%m-%d")

            # Determine COM collection (COM4 or COM5)
            com_type = com_collection.find_one({product_name: {"$exists": True}}, {product_name: 1})
            if not com_type:
                continue
            com_collection_name = com_type[product_name]
            # print(f"Checking date: {date}")
            # print(f"Product: {product_name}, COM Collection: {com_collection_name}")
            # check if sensor data is already in cache
            if (date, product_name) in sensor_data_cache:
                result.append({**harvest, **sensor_data_cache[(date, product_name)]})
                continue
            # Get sensor data from the corresponding COM collection
            sensor_data = db[com_collection_name].find({
                "Datetime": {"$gte": date, "$lt": date + timedelta(days=1)}
            })

            # # export sensor_data to csv
            # print(f"Number of sensor data: {len(list(sensor_data))}")
            sensor_df = pd.DataFrame(sensor_data)
            # sensor_df.to_csv(f'{product_name}.csv', index=False, encoding='utf-8-sig')
            # print(sensor_df.columns)
            # print(sensor_df.head())
            if sensor_df.empty:
                print(f"Empty sensor data for {product_name} on {date}")
                # keep appending to result without sensor data (only harvest data)
                union_entry = harvest
            else:
                avg = sensor_df.drop(["_id", "Datetime"], axis=1).apply(custom_avg).fillna(0)
                # print(avg)
    
                # # Aggregate sensor data (example: calculating averages)
                # aggregated_data = {}
                # 
                # for sensor in sensor_data:
                #     for key, value in sensor.items():
                #         # print(key, value)
                #         if key not in ["_id", "Datetime"]:
                #             if value is None:
                #                 # value = 0
                #                 print(f"Skipping {key} due to None value: {value}")
                #                 continue
                #             aggregated_data[key] = aggregated_data.get(key, 0) + value
                #     count += 1
                # print(f"Count: {count}")
                # # Apply aggregation logic
                # if aggregation == "AVG" and count > 0:
                #     aggregated_data = {k: v / count for k, v in aggregated_data.items()}
                # elif aggregation == "SUM":
                #     pass  # Keep as-is for SUM
    
                # Combine with HARVEST_DATA
                aggregated_data = avg.to_dict()
                sensor_data_cache[(date, product_name)] = aggregated_data
                union_entry = {**harvest, **aggregated_data}
            result.append(union_entry)

            # break

        return result

    # Run the union operation
    final_data = union_data()
    client.close()


Empty sensor data for 메리퀸 on 2024-11-29 00:00:00
Empty sensor data for 죽향저장 on 2024-11-29 00:00:00


In [53]:
# Convert to DataFrame 
df = pd.DataFrame(final_data)
df

,_id,Date,Product Name,Section,Plant,Fruit vertical length,Fruit horizontal length,Fruit weight,Hardness,Sweetness,...,Color b,Remarks,EC,Humidity,Sun,pH,Solid_Temperature,Temperature,CO2,Solid_Moisture
0,676cceac100445d73997654a,2024-06-24,메리퀸,2,3,38.53,28.75,10.98,NaN,16.1,...,27.21,과숙,0.000000,75.509244,15577.452953,0.000000,0.0,20.254857,860.370872,889.453125
1,676cceac100445d73997654b,2024-06-24,메리퀸,3,1,42.76,32.61,15.72,210.0,11.8,...,29.74,NaN,0.000000,75.509244,15577.452953,0.000000,0.0,20.254857,860.370872,889.453125
2,676cceac100445d73997654c,2024-06-24,메리퀸,4,1,NaN,NaN,7.29,NaN,NaN,...,28.51,NaN,0.000000,75.509244,15577.452953,0.000000,0.0,20.254857,860.370872,889.453125
3,676cceac100445d73997654d,2024-06-24,메리퀸,4,2,NaN,NaN,5.22,NaN,NaN,...,29.35,NaN,0.000000,75.509244,15577.452953,0.000000,0.0,20.254857,860.370872,889.453125
4,676cceac100445d73997654e,2024-06-24,메리퀸,4,3,NaN,NaN,3.63,NaN,NaN,...,NaN,NaN,0.000000,75.509244,15577.452953,0.000000,0.0,20.254857,860.370872,889.453125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425,676ccecd100445d739976715,2024-12-06,죽향저장,3,3,50.34,34.96,23.89,150.5,12.7,...,27.50,5,1.269949,81.329227,2986.958743,0.932005,0.0,19.015062,728.586070,55.799999
426,676ccecd100445d739976716,2024-12-06,죽향저장,3,4,43.40,33.44,20.02,179.2,11.6,...,28.57,5,1.269949,81.329227,2986.958743,0.932005,0.0,19.015062,728.586070,55.799999
427,676ccecd100445d739976717,2024-12-06,죽향저장,4,1,NaN,NaN,4.86,NaN,NaN,...,NaN,5,1.269949,81.329227,2986.958743,0.932005,0.0,19.015062,728.586070,55.799999
428,676ccecd100445d739976718,2024-12-06,죽향저장,4,2,40.02,26.02,11.24,161.5,10.4,...,25.87,5,1.269949,81.329227,2986.958743,0.932005,0.0,19.015062,728.586070,55.799999


In [58]:
# sort by Date, Product Name, Section, Plant
df = df.sort_values(by=["Date", "Product Name", "Section", "Plant"])
df

,Date,Product Name,Section,Plant,Fruit vertical length,Fruit horizontal length,Fruit weight,Hardness,Sweetness,Acidity,...,Color b,Remarks,EC,Humidity,Sun,pH,Solid_Temperature,Temperature,CO2,Solid_Moisture
0,2024-06-24,메리퀸,2,3,38.53,28.75,10.98,NaN,16.1,0.75,...,27.21,과숙,0.000000,75.509244,15577.452953,0.000000,0.0,20.254857,860.370872,889.453125
1,2024-06-24,메리퀸,3,1,42.76,32.61,15.72,210.0,11.8,0.63,...,29.74,NaN,0.000000,75.509244,15577.452953,0.000000,0.0,20.254857,860.370872,889.453125
2,2024-06-24,메리퀸,4,1,NaN,NaN,7.29,NaN,NaN,NaN,...,28.51,NaN,0.000000,75.509244,15577.452953,0.000000,0.0,20.254857,860.370872,889.453125
3,2024-06-24,메리퀸,4,2,NaN,NaN,5.22,NaN,NaN,NaN,...,29.35,NaN,0.000000,75.509244,15577.452953,0.000000,0.0,20.254857,860.370872,889.453125
4,2024-06-24,메리퀸,4,3,NaN,NaN,3.63,NaN,NaN,NaN,...,NaN,NaN,0.000000,75.509244,15577.452953,0.000000,0.0,20.254857,860.370872,889.453125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425,2024-12-06,죽향저장,3,3,50.34,34.96,23.89,150.5,12.7,0.62,...,27.50,5,1.269949,81.329227,2986.958743,0.932005,0.0,19.015062,728.586070,55.799999
426,2024-12-06,죽향저장,3,4,43.40,33.44,20.02,179.2,11.6,0.58,...,28.57,5,1.269949,81.329227,2986.958743,0.932005,0.0,19.015062,728.586070,55.799999
427,2024-12-06,죽향저장,4,1,NaN,NaN,4.86,NaN,NaN,NaN,...,NaN,5,1.269949,81.329227,2986.958743,0.932005,0.0,19.015062,728.586070,55.799999
428,2024-12-06,죽향저장,4,2,40.02,26.02,11.24,161.5,10.4,0.62,...,25.87,5,1.269949,81.329227,2986.958743,0.932005,0.0,19.015062,728.586070,55.799999


In [56]:
df.drop(["_id"], axis=1, inplace=True)

In [59]:
df.to_csv('final_data.csv', index=False, encoding='utf-8-sig')

In [57]:
print(df.iloc[0])

Date                         2024-06-24
Product Name                        메리퀸
Section                               2
Plant                                 3
Fruit vertical length             38.53
Fruit horizontal length           28.75
Fruit weight                      10.98
Hardness                            NaN
Sweetness                          16.1
Acidity                            0.75
Color L                           43.82
Color a                           44.52
Color b                           27.21
Remarks                              과숙
EC                                  0.0
Humidity                      75.509244
Sun                        15577.452953
pH                                  0.0
Solid_Temperature                   0.0
Temperature                   20.254857
CO2                          860.370872
Solid_Moisture               889.453125
Name: 0, dtype: object
